## The Street View House Numbers (SVHN) Dataset

**SVHN is a real-world image dataset for developing machine learning and object recognition algorithms with minimal requirement on data formatting but comes from a significantly harder, unsolved, real world problem (recognizing digits and numbers in natural scene images). SVHN is obtained from house numbers in Google Street View images.**


**The goal of this project is to take an image from the SVHN dataset and determine what that digit is. This is a multi-class classification problem with 10 classes, one for each digit 0-9. Digit '1' has label 1, '9' has label 9 and '0' has label 10.**

*The objective of the project is to learn how to implement a simple image classification
pipeline based on the k-Nearest Neighbour and a deep neural network. The goals of this
assignment are as follows:*

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import mpl_toolkits.axes_grid1
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
import h5py
from sklearn.model_selection import GridSearchCV
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = h5py.File('/content/drive/My Drive/SVHN_single_grey1.h5', 'r')

In [4]:
print(list(data.keys()))

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']


In [0]:
X_train = data['X_train'][:]
Y_train=data['y_train'][:]
X_test=data['X_test'][:]
Y_test=data['y_test'][:]
X_val=data['X_val'][:]
Y_val=data['y_val'][:]

In [0]:
data.close()

**1. Data fetching and understand the train/val/test splits.**

In [7]:
print(f'Trainingset X: {X_train.shape}\n Trainingset Y: {Y_train.shape}\nTest set X: {X_test.shape}\n Test set Y: {Y_test.shape}\nValidation set X: {X_val.shape}\n Validation set Y: {Y_val.shape}')

Trainingset X: (42000, 32, 32)
 Trainingset Y: (42000,)
Test set X: (18000, 32, 32)
 Test set Y: (18000,)
Validation set X: (60000, 32, 32)
 Validation set Y: (60000,)


In [ ]:
# Print first few images in test dataset
fig = plt.figure(1, figsize=(10, 10))
grid = mpl_toolkits.axes_grid1.ImageGrid(fig, 111, nrows_ncols=(5, 10), 
                                          axes_pad=0.05)
for img in X_train:
    ax = grid[i]
    img = read_image(filepath, (32, 32))
    ax.imshow(img.astype(np.uint8))
    ax.axis('off')
    i += 1
plt.show();

**2. Implement and apply an optimal k-Nearest Neighbor (kNN) classifier (7.5 points)**

In [0]:
# kmax = 200
kmax = 20
krange = np.arange(1, kmax, 2)
acc = []
for n in krange:
    NNH = KNeighborsClassifier(n_neighbors= n)
    NNH.fit(np.reshape(X_train, (X_train.shape[0],X_train.shape[1]*X_train.shape[2])), Y_train)
    acc.append({'K_value': n, 'accuracy': NNH.score(np.reshape(X_test, (X_test.shape[0],X_test.shape[1]*X_test.shape[2])), Y_test)})

In [0]:
acc = sorted(acc, key = lambda i: i['accuracy'],reverse=True)

In [0]:
print(f"Top accuracy: {acc[0]['accuracy']}\nK-Value: {acc[0]['K_value']}")

Top accuracy: 0.5287222222222222
K-Value: 17


In [9]:
knn_mdl = KNeighborsClassifier(n_neighbors= acc[0]['K_value'])
knn_mdl.fit(np.reshape(X_train, (X_train.shape[0],X_train.shape[1]*X_train.shape[2])), Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=17, p=2,
                     weights='uniform')

In [0]:
y_pred = knn_mdl.predict(np.reshape(X_test, (X_test.shape[0],X_test.shape[1]*X_test.shape[2])))

**3. Print the classification metric report (2.5 points)**

In [13]:
target_names = ['class 10', 'class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7', 'class 8', 'class 9']
print(classification_report(Y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    class 10       0.45      0.70      0.55      1814
     class 1       0.46      0.73      0.56      1828
     class 2       0.62      0.55      0.59      1803
     class 3       0.45      0.43      0.44      1719
     class 4       0.67      0.65      0.66      1812
     class 5       0.51      0.39      0.44      1768
     class 6       0.50      0.41      0.45      1832
     class 7       0.71      0.62      0.67      1808
     class 8       0.47      0.36      0.41      1812
     class 9       0.55      0.43      0.48      1804

    accuracy                           0.53     18000
   macro avg       0.54      0.53      0.52     18000
weighted avg       0.54      0.53      0.52     18000



**4. Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations) (5 points)**

**5. Implement batch normalization for training the neural network (2.5 points)**

In [14]:
model = tf.keras.models.Sequential()
# Reshape data from 2D to 1D -> 32x32 to 1024
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))

# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10000, activation='relu'))

model.add(tf.keras.layers.Dense(10000, activation='relu'))

model.add(tf.keras.layers.Dense(10, activation='relu'))



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


**6. Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions) (2.5 points)**

In [0]:
# sgd Optimizer
sgd = tf.keras.optimizers.SGD(lr=0.01)
# Compile the model
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]
y_train = tf.keras.utils.to_categorical(Y_train, 10)
# model.fit(X_train, Y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=50, callbacks=callback_list)

model.fit(X_train, y_train, batch_size=16, epochs=50, callbacks=callback_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 42000 samples
Epoch 1/50
42000/42000 [==============================] - 1754s 42ms/sample - loss: 5.0058 - acc: 0.1520
Epoch 2/50
42000/42000 [==============================] - 1731s 41ms/sample - loss: 3.5400 - acc: 0.1828
Epoch 3/50
42000/42000 [==============================] - 1737s 41ms/sample - loss: 3.4828 - acc: 0.1855
Epoch 4/50
42000/42000 [==============================] - 1731s 41ms/sample - loss: 3.4300 - acc: 0.2316
Epoch 5/50
42000/42000 [==============================] - 1732s 41ms/sample - loss: 3.2988 - acc: 0.3123
Epoch 6/50
42000/42000 [==============================] - 1733s 41ms/sample - loss: 2.2434 - acc: 0.1938
Epoch 7/50
42000/42000 [==============================] - 1730s 41ms/sample - loss: 2.0475 - acc: 0.2564
Epoch 8/50
42000/42000 [==============================] - 1722s 41ms/sample - loss: 2.0294 - acc: 0.2456
Epoch 9/50
42000/42000 [===================

**7. Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics (5 points)**

In [20]:
# Final evaluation of the model
y_test = tf.keras.utils.to_categorical(Y_test, 10)

test_scores = model.evaluate(X_test,y_test, verbose=0)
print(f"Test Set Accuracy: {test_scores[1]*100}")

Test Set Accuracy: 10.066666454076767


In [ ]:
batch_size = [10, 20, 40, 60, 80]
epochs = [10, 15, 20, 25, 30, 40, 50]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=3)
grid_result = grid.fit(X, y)

* The knn model is comparatively time consuming than the neural network model. It is also resource consuming which has been resulting in the very longer training time as well as the prediction times.
* The accuracy reached with such a process doesn't do justice
* On the other hand the neural network implementation is comparatively easy and the predictin of values is also lesser time consuming than the KNN model
* grid search implementation can considerable increase the accuracy of the neural network model